In [ ]:
import re
import numpy as np
import json
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
p1=re.compile("Longest Blockchain Length: (\d*)")
p2=re.compile("generate a block with parent height: (\d*)")

def process_one(filename):
    last_longest = None
    attack_forking_start = None
    attack_success = None
    with open(filename) as fin:
        for i, line in enumerate(fin):
            result = p1.search(line)
            if result is not None:
                last_longest = (i, int(result.group(1)) )
            result = p2.search(line)
            if result is not None:
                parent = int(result.group(1))
                if attack_forking_start is None:
                    attack_forking_start = parent
                attack_forking_length = parent+1
                if attack_forking_length >= last_longest[1]:
                    #attack success
                    attack_success = attack_forking_length
    return attack_forking_start, attack_success

In [ ]:
def generate_matrix(filepath):
    matrix = np.zeros((11,11))
    for betas in range(11):
        for betaw in range(11):
            if betas+betaw==0:
                continue
            b1=betas/10
            b2=betaw/10
            attack_forking_start, attack_success = process_one(filepath+str(betas)+'-'+str(betaw)+'-attacker.log')
            if attack_success is not None:
                matrix[betas][betaw] = attack_success-attack_forking_start
    return matrix

In [ ]:
def generate_array(filepath):
    # for pure pos / pow
    matrix = np.zeros((11,))
    for betas in range(11):
        if betas==0:
            continue
        b1=betas/10
        attack_forking_start, attack_success = process_one(filepath+str(betas)+'-attacker.log')
        if attack_success is not None:
            matrix[betas] = attack_success-attack_forking_start
    return matrix

In [ ]:
minotaur = generate_matrix('')
pure_pos = generate_array('')

In [ ]:
minotaur_max = np.ceil(np.max(minotaur))
norm = matplotlib.colors.Normalize(vmin=0, vmax=minotaur_max)

grid = dict(height_ratios=[minotaur.shape[0],1], width_ratios=[1,minotaur.shape[0], 0.5 ])
fig, axes = plt.subplots(ncols=3, nrows=2, gridspec_kw = grid)
axes[0,1].imshow(minotaur, cmap='Reds', aspect='equal', interpolation='nearest', origin='lower', norm=norm)
axes[0,0].imshow(pure_pos.reshape((11,1)), cmap='Reds', aspect='equal', interpolation='nearest', origin='lower', norm=norm)
axes[1,1].imshow(pure_pos.reshape((1,11)), cmap='Reds', aspect='equal', interpolation='nearest', origin='lower', norm=norm)
axes[1,0].axis("off")
axes[1,2].axis("off")
for ax in [axes[0,1], axes[0,0], axes[1,1]]:
    ax.set_xticks([]); ax.set_yticks([])
fig.colorbar(matplotlib.cm.ScalarMappable(cmap="Reds", norm=norm), cax=axes[0,2])